# Set-up

## Libraries and Dashboard

In [6]:
# Import Libraries
%load_ext autoreload
%autoreload

import os
#if not os.path.basename(os.getcwd()) == "datenguide-python": os.chdir("..")
    
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
#import glob
import geopandas as gpd
#import geojsonio
#import ipywidgets as widgets
#from ipywidgets import interact, interactive, fixed, interact_manual
import math
from IPython.display import Image

# Plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
#import cufflinks as cf

# Datenguidepy
from datenguidepy.query_helper import get_regions, get_statistics, get_availability_summary
from datenguidepy import Query

# Processing/App
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Dashboard

#Global variables
regions = get_regions()
statistics = get_statistics()
availability_summary = get_availability_summary()  # Save availability summary as dataframe
availability_summary.reset_index(inplace=True)
nuts3_codes = pd.DataFrame(get_regions().query('level == "nuts3"').name)   # Get all NUTS-3 codes
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]       # Define years for timeframe

#Table display settings
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 50)

#Plotting Settings
#sns.set(style="white")  #, palette=["#FF5A5F", "#00A699"])
#sns.set_palette(
#    sns.color_palette(
#        ["#00A699", "#767676", "#FF5A5F", "#F9C70C", "#FC642D", "#484848"]))

pio.templates.default = "simple_white"
px.defaults.template = "simple_white"  # 'ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none'
px.defaults.color_continuous_scale = [
    "#00A699", "#50D6B9", "#FF9A7F", "#FF5A5F"
]  # ["#50D6B9", "#00A699", "#FF9A7F", "#FF5A5F"] #"burgyl"
px.defaults.color_discrete_sequence = [
    "#00A699", "#767676", "#FF5A5F", "#F9C70C", "#FC642D", "#484848"
]
px.defaults.width = 900
px.defaults.height = 600

**Global functions**

In [9]:
# Function for standardized query (I get duplicate rows, so I delete them in sequence. Can surely be improved)
def nuts3_query(code):
    query = Query.region(list(nuts3_codes.index))    # Query.all_regions(nuts=3) did not work for me somehow
    query.add_field(code)
    query_res = query.results(verbose_statistics=True)    # verbose_statistics changes column name of code to title
    value_col = [col for col in query_res.columns if code in col][0]   # necessary as "code" is no longer column title
    query_res = query_res[["id", "name", "year", value_col]]     # retain only required columns
    query_res.drop_duplicates(subset=None, keep='first', inplace=True)
    query_res.rename(columns=lambda x: x+"_nuts3" if x in ["id", "name"] else x, inplace=True)
    return query_res

## Dataset contents

In [4]:
# Display regions
regions

,name,level,parent
region_id,,,
10,Saarland,nuts1,DG
11,Berlin,nuts1,DG
12,Brandenburg,nuts1,DG
13,Mecklenburg-Vorpommern,nuts1,DG
14,Sachsen,nuts1,DG
15,Sachsen-Anhalt,nuts1,DG
16,Thüringen,nuts1,DG
100,Saarland,nuts2,10
110,Berlin,nuts2,11


In [5]:
# Query regions, e.g. get all "Bundesländer
get_regions().query("level == 'nuts1'")

,name,level,parent
region_id,,,
10,Saarland,nuts1,DG
11,Berlin,nuts1,DG
12,Brandenburg,nuts1,DG
13,Mecklenburg-Vorpommern,nuts1,DG
14,Sachsen,nuts1,DG
15,Sachsen-Anhalt,nuts1,DG
16,Thüringen,nuts1,DG
01,Schleswig-Holstein,nuts1,DG
02,Hamburg,nuts1,DG


In [6]:
# Display all statistics
statistics

,short_description,long_description
statistic,,
RLE001,Regelleistungsempfänger,Regelleistungsempfänger\n \n \nErläuterung für folgende Statistik(en):\n22221 Statistik der Empfänger von Asylbewerberregelleis-\n tungen\n \...
BEV002,Gestorbene,Gestorbene\n \n \nErläuterung für folgende Statistik(en):\n12613 Statistik der Sterbefälle\n \nBegriffsinhalt:\nSterbefälle\n \nAls Sterbefälle we...
FLCX05,Bodenfläche,Bodenfläche\n \n \nErläuterung für folgende Statistik(en):\n33111 Flächenerhebung nach Art der tatsächlichen Nutzung\n \nBegriffsinhalt:\nBodenflä...
FLC005,Bodenfläche,Bodenfläche\n\n\nErläuterung für folgende Statistik(en):\n33111 Flächenerhebung nach Art der tatsächlichen Nutzung\n \nBegriffsinhalt:\nBodenfläch...
KIND35,Kinder,Missing
KIND36,Betreuungsquote,Betreuungsquote\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik der Kinder und tätigen Personen in Ta-\n geseinri...
KIND37,Anteil der Kinder mit ausl. Herkunft mind. eines Elternteils,Anteil der Kinder mit ausländischer Herkunft mindestens\neines Elternteils\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik d...
KIND38,Ant.d.Kinder m.e.durchgeh. Betreuungszeit v.m.a. 7 Std.p.Betreuungstag,Anteil der Kinder mit einer durchgehenden Betreuungszeit von\nmehr als 7 Stunden pro Betreuungstag\n \n \nErläuterung für folgende Statistik(en):\...
PERS01,"Pädagog.,Leitungs-u.Verwaltg.personal i.Kita.einr.","Pädagogisches, Leitungs- und Verwaltungspersonal in Kinder-\ntageseinrichtungen\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statis..."


In [7]:
# Find out the name of the desired statistic about birth
get_statistics().query('long_description.str.contains("Statistik der Geburten")', engine='python')

,short_description,long_description
statistic,,
AI0209,Lebendgeborene je 10.000 Einwohner,"wiki\n==Lebendgeborene je 10.000 Einwohner==\n===Aussage===\nDer Indikator gibt Auskunft darüber, wie viele Neugeborene\npro Jahr auf 10.000 Einwo..."
AI0211,Geburten-/Gestorbenenüberschuss je 10.000 Einw.,wiki\n==Geburtenüberschuss (+) bzw. Gestorbenenüberschuss (-) je\n10.000 Einwohner==\n===Aussage===\nDer Geburtensaldo bildet die natürliche Bevöl...
AI0219,Durchschnittsalter der Mutter bei der Geburt des 1. Kindes,wiki\n==Durchschnittsalter der Mutter bei der Geburt des 1.\nlebendgeborenen Kindes==\n\n===Aussage=== \nDer Indikator gibt Auskunft über das durc...
BEV001,Lebend Geborene,Lebend Geborene\n \n \nErläuterung für folgende Statistik(en):\n12612 Statistik der Geburten\n \nBegriffsinhalt:\nLebendgeborene\n \nLebendgeboren...
BEVDG1,Durchschnittsalter der Mutter bei der Geburt,Durchschnittsalter der Mutter bei der Geburt\n\n\nErläuterung für folgende Statistik(en):\n12612 Statistik der Geburten\n \nBegriffsinhalt:\nDurch...


## General demographic data for Germany (regional)

- Population per region
- Children per age per region
- ...

# Data inspection

## Via Ipywidgets Dashboard

In [8]:
# Extract statistics_tuple
statistics_tuple = get_statistics().reset_index().loc[:,["short_description", "statistic"]]
statistics_tuple = sorted([tuple(x) for x in statistics_tuple.to_numpy()])

In [9]:
# Define nuts_selector
def nuts_selector(nuts_level):
    
    regions = get_regions().query(f"level == {nuts_level}").reset_index()[["name", "region_id"]]
    regions = sorted([tuple(x) for x in regions.to_numpy()])

    @interact
    def get_statistics_dashboard(region=regions, statistic=statistics_tuple):

        q = Query.region(region)

        try:
            field = q.add_field(statistic)
        except KeyError as e:
            return(f"Statistic not available: {e}")
        
        return q.results()

interact(nuts_selector, nuts_level=[('NUTS 1', "'nuts1'"), ('NUTS 2', "'nuts2'"), ('NUTS 3', "'nuts3'")]);

interactive(children=(Dropdown(description='nuts_level', options=(('NUTS 1', "'nuts1'"), ('NUTS 2', "'nuts2'")…

## Via Keyword Search Query

In [10]:
# Search get_statistics('KEYWORD')
get_statistics('kind')

,short_description,long_description
statistic,,
KIND35,Kinder,Missing
KIND37,Anteil der Kinder mit ausl. Herkunft mind. eines Elternteils,Anteil der Kinder mit ausländischer Herkunft mindestens\neines Elternteils\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik d...
KIND38,Ant.d.Kinder m.e.durchgeh. Betreuungszeit v.m.a. 7 Std.p.Betreuungstag,Anteil der Kinder mit einer durchgehenden Betreuungszeit von\nmehr als 7 Stunden pro Betreuungstag\n \n \nErläuterung für folgende Statistik(en):\...
PERS02,Kindertagespflegepersonen,Kindertagespflegepersonen\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik der Kinder und tätigen Personen in Ta-\n ...
KIND01,Betreute Kinder,Missing
KIND33,Betreute Kinder von unter 3 Jahren,Missing
KIND34,Betreute Kinder von unter 14 Jahren,Missing
KIND02,"Kinder, Herkunft mind. 1 Elternteil ausländisch","Kinder, Herkunft mindestens 1 Elternteil ausländisch\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik der Kinder und tätigen ..."
KIND03,Kinder mit Mittagsverpflegung,Kinder mit Mittagsverpflegung\n \n \nErläuterung für folgende Statistik(en):\n22541/22543 Statistik der Kinder und tätigen Personen in Ta-\n ...


In [11]:
# Search availability_summary('CODE')
availability_summary[availability_summary.statistic == "PERS02"]

,region_id,statistic,region_name,entries,start_year,end_year
65,10,PERS02,Saarland,25,2007.0,2019.0
95,11,PERS02,Berlin,25,2007.0,2019.0
125,12,PERS02,Brandenburg,25,2007.0,2019.0
155,13,PERS02,Mecklenburg-Vorpommern,25,2007.0,2019.0
185,14,PERS02,Sachsen,25,2007.0,2019.0
215,15,PERS02,Sachsen-Anhalt,25,2007.0,2019.0
245,16,PERS02,Thüringen,25,2007.0,2019.0
275,01,PERS02,Schleswig-Holstein,25,2007.0,2019.0
305,02,PERS02,Hamburg,25,2007.0,2019.0
335,03,PERS02,Niedersachsen,25,2007.0,2019.0


## Via Code Query

In [12]:
# Define parameters for query
#regions = get_regions()
#ids = list(regions.index) # Alt.: "01", list(regions[regions.parent =="130"].index)

In [13]:
# Query region and code
#query = Query.all_regions(nuts=1)
#field = query.add_field('AI1801')
#result_df = query.results(verbose_statistics=True)
#result_df.sort_values('year')

In [14]:
# get information on the field
#field_info = field
#field_info = query.add_field('AI1801')
#field_info.get_info()

# Analysis

In [15]:
# Get all NUTS-3 codes and define timeframe
nuts3_codes = pd.DataFrame(get_regions().query('level == "nuts3"').name)
#years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [16]:
# Create default dataframe on NUTS-3 level (2010-2019)
#df_template = pd.concat([nuts3_codes, pd.DataFrame(columns=years)])

## Material well-being

### NEW: 2014 indicator of progressive behaviour towards role of women/men as parents .....prior: <s>percentage of children in families without an employed adult</s>

In [17]:
# Get relevant data
#query1 = Query.all_regions(nuts=1)
#query1.add_field('AI1801')
#result_df1 = query.results(verbose_statistics=True)
#result_df1 = result_df1[result_df1.year == 2014].sort_values("Elterngeldbezug Vater (AI1801)", ascending=False)

In [18]:
# Plot comparison
#fig = px.bar(result_df1, x='name', y='Elterngeldbezug Vater (AI1801)', title="Anteil Väter an Elterngeldbezug (in %)")
#fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=True)
#fig.show()

In [19]:
# Get data on NUTS3 level
mwb_prog = nuts3_query('AI1801')

In [20]:
# Get additional information on the field
query = Query.all_regions(nuts=1)
mwb_prog_info = query.add_field('AI0304')
mwb_prog_info.get_info()

kind:
OBJECT

description:
Anteil Schulabgänger mit allgem. Hochschulreife

arguments:
year: LIST of type SCALAR(Int)

statistics: LIST of type ENUM(AI0304Statistics)
enum values:
R99910: Regionalatlas Deutschland

fields:
id: Interne eindeutige ID
year: Jahr des Stichtages
value: Wert
source: Quellenverweis zur GENESIS Regionaldatenbank

enum values:
None


In [21]:
# Save DataFrame
mwb_prog.to_pickle("data_pickles/share_fathers_with_parental_allowance.pkl")

## Health and well-being

### NEW: Percentage of children leaving school with "allgemeine Hochschulreife" (not included in map)

**VERSION 1: id and year as single columns, one value column**

In [ ]:
# Get content through query
health_1 = nuts3_query('AI0304')

In [ ]:
health_1.head()

In [ ]:
health_1 = health_1.rename(columns={"Unnamed: 2": "id_nuts3"})

In [24]:
# Save DataFrame
health_1.to_pickle("data_pickles/schulabg_mit_hochschulreife.pkl")

### NEW: Percentage of children leaving school without certificate .....prior: <s>percentage aged 15-19 remaining in education</s> (not included in map)

**Anteil der Schulabgänger/-innen allgemeinbildender Schulen ohne Hauptschulabschluss an den Schulabgänger/-innen allgemeinbildender Schulen insgesamt**

**Aussage**
Der Indikator gibt an, wie viel Prozent aller Schulabgänger die Schule vorzeitig bzw. ohne einen Abschluss beenden und damit die ungünstigsten Voraussetzungen für den Einstieg in die Berufsausbildung haben.

**Indikatorberechnung**
Für die Berechnung des Indikators Anteil der Schulabgänger/-innen allgemeinbildender Schulen ohne Hauptschulabschluss an den Schulabgänger/-innen allgemeinbildender Schulen insgesamt wird die Zahl der Schulabgänger/-innen allgemeinbildender Schulen ohne Hauptschulabschluss im Jahr durch die Zahl aller Schulabgänger/-innen allgemeinbildender Schulen insgesamt im Jahr dividiert und mit 100 multipliziert.

**Herkunftsstatistiken**
Der Indikator beruht auf Daten der Statistik der allgemeinbildenden Schulen.

**Merkmalsbeschreibungen**
Schulen
- Im weitesten Sinne gilt als Schule eine Bildungsstätte, -einrichtung oder -anstalt, in der Unterricht nach einem von der zuständigen Aufsichtsbehörde festgesetzten oder genehmigten Lehrplan erteilt wird.
- Absolventen/Abgänger ohne Hauptschulabschluss
- Hierzu zählen Absolventen/Abgänger aus Haupt-/Volksschulen, Förderschulen, Schulen mit mehreren Bildungsgängen, Gesamtschulen ohne Hauptschulabschluss, sowie Abgänger aus Klassen-/Jahrgangsstufe 7 und 8 (bei Ländern mit 10jähriger Vollzeitschulpflicht auch aus Klassen-/Jahrgangsstufe 9) der Realschulen, Gymnasien, Freien Waldorfschulen und drei- und vierstufigen Wirtschaftsschulen, nach Ableistung der allgemeinen Vollzeitschulpflicht.

In [25]:
# Get content through query
health_2 = nuts3_query('AI0305')

In [26]:
# Save DataFrame
health_2.to_pickle("data_pickles/schulabg_ohne_hauptschulabschl.pkl")

## Behaviours and risks

### NEW: 2019 Percentage of <14 and <18yr olds accused of serious assault involving heavy injuries.....prior: <s>percentage of 11, 13 and 15 yearolds involved in fighting in last 12 months</s> (not included in map)

In [27]:
# Get relevant data
#query2 = Query.all_regions(nuts=1)
#query2.add_field('AI1801')
#result_df2 = query.results(verbose_statistics=True)
#result_df2 = result_df2[result_df2.year == 2014].sort_values("id")

Data of alleged crimes from: [BKA 2019](https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/PKS2019/InteraktiveKarten/06GefaehrlicheSchwereKoerperverletzung/06_GefaehrlicheSchwereKoerperverletzung_node.html) (inputted manually, as only available as PDF)

In [28]:
# Manually add data of alleged crimes to dataframe
#result_df2["children_u14_m"] = [165, 325, 704, 50, 1567, 316, 288, 439, 638, 58, 507, 250, 181, 266, 212, 122]
#result_df2["children_u14_f"] = [27, 76, 136, 26, 395, 46, 92, 111, 153, 7, 104, 31, 49, 38, 45, 37]
#result_df2["young_u18_m"] = [447, 407, 1621, 211, 4011, 1077, 764, 1538, 1723, 214, 932, 460, 378, 657, 493, 325]
#result_df2["young_u18_f"] = [90, 127, 361, 54, 993, 199, 199, 267, 313, 52, 174, 81, 75, 120, 85, 43]

In [29]:
# Display result
#result_df2

## Additional to UNICEF index

### Institutions for child welfare/ caretaking, such as the availability of Kitas, sport clubs or educational support (not included in map)

In [30]:
# Get relevant data
result_df3 = pd.read_csv("data/einrichtungen_regional.csv", encoding='unicode_escape', delimiter=";")

### Crime suspects in NUTS-3 regions

**Description "Crime suspects <14 per 1000"**

Number of crime suspects from 0-13 (incl.) years in given NUTS-3 region. Includes all possible types of crimes (please refer to this [list](https://www.bka.de/SharedDocs/Downloads/DE/Publikationen/PolizeilicheKriminalstatistik/2019/Interpretation/01_div_Dok/Straftatenkatalog-Historie.pdf?__blob=publicationFile&v=3) for a full explanation of included crimes). The value shows the negative of crimes per 1000 population (e.g. -1.4 indicates 1.4 suspects per 1000 population in that NUTS-3 region in given year). Data is taken from [BKA](https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/pks_node.html).

**Description "Crime suspects 14-<18 per 1000"**

Number of crime suspects from 14-17 (incl.) years in given NUTS-3 region. Includes all possible types of crimes (please refer to this [list](https://www.bka.de/SharedDocs/Downloads/DE/Publikationen/PolizeilicheKriminalstatistik/2019/Interpretation/01_div_Dok/Straftatenkatalog-Historie.pdf?__blob=publicationFile&v=3) for a full explanation of included crimes). The value shows the negative of crimes per 1000 population (e.g. -1.4 indicates 1.4 suspects per 1000 population in that NUTS-3 region in given year). Data is taken from [BKA](https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/pks_node.html).

**Get data for suspected criminals and merge to one DataFrame**

In [165]:
# Dataset for 2014
suspects_2014 = pd.read_csv("data/tatverdaechtige_2014.csv", encoding='unicode_escape', delimiter=";", decimal=',')
suspects_2014 = suspects_2014[(suspects_2014["2"] == "Straftaten insgesamt") & (suspects_2014["3"] == "X")][["Unnamed: 2", "10", "13"]]
suspects_2014 = suspects_2014.rename(columns={"Unnamed: 2": "id_nuts3","10": "Crime suspects <14 per 1000", "13": "Crime suspects 14-<18 per 1000"})
suspects_2014["year"] = "2014"

In [166]:
# Loop for remaining years
suspects_all = suspects_2014.copy()
for year in ["2015", "2016", "2017", "2018"]:
    suspects_year = pd.read_csv(f"data/tatverdaechtige_{year}.csv", encoding='unicode_escape', delimiter=";", decimal=',')
    suspects_year = suspects_year[(suspects_year["2"] == "Straftaten insgesamt") & (suspects_year["6"] == "X")][["3", "13", "16"]]
    suspects_year = suspects_year.rename(columns={"3": "id_nuts3","13": "Crime suspects <14 per 1000", "16": "Crime suspects 14-<18 per 1000"})
    suspects_year["year"] = year
    suspects_all = suspects_all.append(suspects_year, ignore_index=False, verify_integrity=False, sort=None)


C:\Users\Mauricio\anaconda3\envs\airbnb42\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning:

Columns (0,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.



In [167]:
# Convert year to int
suspects_all.year = [int(year) for year in suspects_all.year]

In [168]:
# Add "0" to 4-digit NUTS-3 id
suspects_all.id_nuts3 = ["0"+el if len(el)==4 else el for el in suspects_all.id_nuts3]

**Get base dataframe and population from query**

In [36]:
# Get data on NUTS3 level
straftaten = nuts3_query('BEVSTD')

In [170]:
# Assign dataset
straftaten = straftaten_raw

In [171]:
# Convert year and BEVSTD to int
straftaten.year = straftaten.year.astype(int)
straftaten["Bevölkerungsstand (BEVSTD)"] = straftaten["Bevölkerungsstand (BEVSTD)"].astype(int)

In [172]:
# Keep only 2014-2018
straftaten = straftaten[straftaten.year > 2013]

In [173]:
# Remove duplicates
straftaten = straftaten.drop_duplicates(["id_nuts3", "year"], keep='last')

In [174]:
# Sort values
straftaten = straftaten.sort_values("id_nuts3")

**Merge datasets and bring in format**

In [201]:
# Merge datasets
result_df4 = pd.merge(straftaten, suspects_all, how='left', left_on=["id_nuts3", "year"], right_on=["id_nuts3", "year"])

In [202]:
# Convert values to string
result_df4["Crime suspects <14 per 1000"] = result_df4["Crime suspects <14 per 1000"].astype(str)
result_df4["Crime suspects 14-<18 per 1000"] = result_df4["Crime suspects 14-<18 per 1000"].astype(str)

In [203]:
# Replace ","
result_df4["Crime suspects <14 per 1000"] = result_df4["Crime suspects <14 per 1000"].apply(lambda x: x.replace(',',''))
result_df4["Crime suspects 14-<18 per 1000"] = result_df4["Crime suspects 14-<18 per 1000"].apply(lambda x: x.replace(',',''))

In [204]:
# Remove nan
result_df4 = result_df4[result_df4["Crime suspects <14 per 1000"]!="nan"]

In [205]:
# Convert back to int
result_df4["Crime suspects <14 per 1000"] = result_df4["Crime suspects <14 per 1000"].astype(int)
result_df4["Crime suspects 14-<18 per 1000"] = result_df4["Crime suspects 14-<18 per 1000"].astype(int)

In [206]:
# Calculate values as per 1000 population
result_df4["Crime suspects <14 per 1000"] = result_df4["Crime suspects <14 per 1000"] / result_df4["Bevölkerungsstand (BEVSTD)"] * -1000
result_df4["Crime suspects 14-<18 per 1000"] = result_df4["Crime suspects 14-<18 per 1000"] / result_df4["Bevölkerungsstand (BEVSTD)"] * -1000

In [210]:
# Drop BEVSTD
result_df4.drop(
        ["Bevölkerungsstand (BEVSTD)"],
        inplace=True,
        axis=1)

In [297]:
# Save DataFrame
result_df4.to_pickle("data_pickles/crime_suspects_per_1000.pkl")

### Crime victims in NUTS-3 regions

**Description "Crime victims <14 per 1000"**

Number of crime victims from 0-13 (incl.) years in given NUTS-3 region. Includes all possible types of crimes (please refer to this [list](https://www.bka.de/SharedDocs/Downloads/DE/Publikationen/PolizeilicheKriminalstatistik/2019/Interpretation/01_div_Dok/Straftatenkatalog-Historie.pdf?__blob=publicationFile&v=3) for a full explanation of included crimes). The value shows the negative of victims per 1000 population (e.g. -1.74 indicates 1.74 victims per 1000 population in that NUTS-3 region in given year). Data is taken from [BKA](https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/pks_node.html).

**Description "Crime victims 14-<18 per 1000"**

Number of crime victims from 14-17 (incl.) years in given NUTS-3 region. Includes all possible types of crimes (please refer to this [list](https://www.bka.de/SharedDocs/Downloads/DE/Publikationen/PolizeilicheKriminalstatistik/2019/Interpretation/01_div_Dok/Straftatenkatalog-Historie.pdf?__blob=publicationFile&v=3) for a full explanation of included crimes). The value shows the negative of victims per 1000 population (e.g. -1.74 indicates 1.74 victims per 1000 population in that NUTS-3 region in given year). Data is taken from [BKA](https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/pks_node.html).

**Get data for suspected criminals and merge to one DataFrame**

In [279]:
# Dataset for 2014
victims_2014 = pd.read_csv("data/opfer_2014.csv", encoding='unicode_escape', delimiter=";", decimal=',')
victims_2014 = victims_2014[(victims_2014["2"] == "Straftaten insgesamt") & (victims_2014["3"] == "insg.")][["Unnamed: 2", "10", "13"]]
victims_2014 = victims_2014.rename(columns={"Unnamed: 2": "id_nuts3", "10": "Crime victims <14 per 1000", "13": "Crime victims 14-<18 per 1000"})
victims_2014["year"] = "2014"

C:\Users\Mauricio\anaconda3\envs\airbnb42\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning:

Columns (0,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.



In [280]:
# Loop for remaining years
victims_all = victims_2014.copy()
for year in ["2015", "2016", "2017", "2018"]:
    victims_year = pd.read_csv(f"data/opfer_{year}.csv", encoding='unicode_escape', delimiter=";", decimal=',')
    victims_year = victims_year[(victims_year["2"] == "Straftaten insgesamt") & (victims_year["6"] == "insg.")][["3", "25", "34"]]
    victims_year = victims_year.rename(columns={"3": "id_nuts3","25": "Crime victims <14 per 1000", "34": "Crime victims 14-<18 per 1000"})
    victims_year["year"] = year
    victims_all = victims_all.append(victims_year, ignore_index=False, verify_integrity=False, sort=None)


In [281]:
# Convert year to int
victims_all.year = [int(year) for year in victims_all.year]

In [282]:
# Add "0" to 4-digit NUTS-3 id
victims_all.id_nuts3 = ["0"+el if len(el)==4 else el for el in victims_all.id_nuts3]

**Get base dataframe and population from query**

In [284]:
# Copy straftaten dataset
opfer = straftaten.copy()

**Merge datasets and bring in format**

In [286]:
# Merge datasets
result_df5 = pd.merge(opfer, victims_all, how='left', left_on=["id_nuts3", "year"], right_on=["id_nuts3", "year"])

In [287]:
# Convert values to string
result_df5["Crime victims <14 per 1000"] = result_df5["Crime victims <14 per 1000"].astype(str)
result_df5["Crime victims 14-<18 per 1000"] = result_df5["Crime victims 14-<18 per 1000"].astype(str)

In [288]:
# Replace ","
result_df5["Crime victims <14 per 1000"] = result_df5["Crime victims <14 per 1000"].apply(lambda x: x.replace(',',''))
result_df5["Crime victims 14-<18 per 1000"] = result_df5["Crime victims 14-<18 per 1000"].apply(lambda x: x.replace(',',''))

In [289]:
# Remove nan
result_df5 = result_df5[result_df5["Crime victims <14 per 1000"]!="nan"]

In [290]:
# Convert back to int
result_df5["Crime victims <14 per 1000"] = result_df5["Crime victims <14 per 1000"].astype(int)
result_df5["Crime victims 14-<18 per 1000"] = result_df5["Crime victims 14-<18 per 1000"].astype(int)

In [291]:
# Calculate values as per 1000 population
result_df5["Crime victims <14 per 1000"] = result_df5["Crime victims <14 per 1000"] / result_df5["Bevölkerungsstand (BEVSTD)"] * -1000
result_df5["Crime victims 14-<18 per 1000"] = result_df5["Crime victims 14-<18 per 1000"] / result_df5["Bevölkerungsstand (BEVSTD)"] * -1000

In [292]:
# Drop BEVSTD
result_df5.drop(
        ["Bevölkerungsstand (BEVSTD)"],
        inplace=True,
        axis=1)

In [298]:
# Save DataFrame
result_df5.to_pickle("data_pickles/crime_victims_per_1000.pkl")